# Введение в анализ данных
## НИУ ВШЭ, 2019-2020 учебный год

### Домашнее задание №3

Задание выполнил(а): *Скворцов Иван*

### Общая информация

__Дата выдачи:__ 08.04.2020

__Дедлайн:__ 23:59 27.04.2020


### Оценивание и штрафы

Оценка за ДЗ вычисляется по следующей формуле:

$$
\min(\text{points}, 21)  \times 10 / 21,
$$

где points — количество баллов за домашнее задание, которое вы набрали. Максимальное число баллов, которое можно получить за решение данного домашнего задания — 24, все баллы сверх 21 идут в бонус (таким образом, за данное домашнее задание можно получить 3 бонусных балла). Накопленные бонусные баллы можно будет потом распределять по другим домашним заданиям и проверочным (+1 бонусный балл = +1 к оценке за домашнее задание/проверочную).

За сдачу задания позже срока на итоговую оценку за задание накладывается штраф в размере 1 балл в день, но получить отрицательную оценку нельзя.

__Внимание!__ Домашнее задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов.

### Формат сдачи

Загрузка файлов с решениями происходит в системе [Anytask](https://anytask.org/).

Инвайт для группы ИАД-4: zG1cIyT

Перед отправкой перезагрузите ноутбук и проверьте, что все ячейки могут быть последовательно выполнены. Ноутбук должен запускаться с использованием python 3.6+

### Подготовка данных

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

In [2]:
# чтобы видеть проход по итерациям, можно использовать библиотеку tqdm
# она работает примерно так:
from tqdm import tqdm
import time

for i in tqdm(range(100)):
    time.sleep(0.1)

100%|██████████| 100/100 [00:10<00:00,  9.70it/s]


### Данные

Мы имеем дело с данными с торговой платформы Avito.
Для каждого товара представлены следующие параметры:
 - `'title'`
 - `'description'`
 - `'Category_name'`
 - `'Category'`

Имеется информация об объектах 50 классов.
Задача: по новым объектам (`'title'`, `'description'`) предсказать `'Category'`.
(Очевидно, что параметр `'Category_name'` для предсказания классов использовать нельзя)

In [3]:
data = pd.read_csv("avito_data.csv", index_col='id')

data.head()

,title,description,Category_name,Category
id,,,,
382220,Прихожая,В хорошем состоянии. Торг,Мебель и интерьер,20
397529,Кордиант 215/55/16 Летние,Кордиант 215/55/16 Летние/\n /\nАртикул: 1737l...,Запчасти и аксессуары,10
584569,Стол,"Стол, 2 рабочих места . Стол серого цвета, в д...",Мебель и интерьер,20
2513100,Комбинезон,Размер-42/44,"Одежда, обувь, аксессуары",27
1091886,Ветровка,На 2 года,Детская одежда и обувь,29


In [4]:
data.shape

(30000, 4)

In [5]:
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()

del data

Сразу разделим выборку на train и test.
Никакие данные из test для обучения использовать нельзя!

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
X_train[:5]

array([['Сапоги 46 размер новые', 'Сапоги 46 размер новые'],
       ['Светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку. В эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iPhone 7 plus 128GB Red красный в наличии',
        '\xa0/\n/\n Данная цена только для подписчиков Instagram: iQmac/\n/\n Новый красный айфон 7 Plus в наличии это элегантный и мощный смартфон, который готов в полной мере раскрыть новые возможности iOS 10. Аппарат с 4-ядерным процессором А10 и 3 ГБ ОЗУ с легкостью решает самые ресурсоемкие задачи, позволяя наслаждаться быстродействием «тяжелых» приложений и игр на 5,5-дюймовом дисплее. Аппарат получил экран, как у iPad Pro, так что картинка теперь соответствует кинематографическому стандарту.'],
       ['Пион Ирис Ромашка рассада',
        'Пион куст 500 р ( более 10 шт)/\nСаженец/ корень 100р/\nРастут у нас более 70 лет/\nРозовые, бордовые и белые/\nНа фото цветы 2018г/\nП. Зубчанинов

In [8]:
y_train[:5]

array([ 27,  20,  84, 106,  27])

### Токенизация (0.5 балла)


Токенизация -- разбиение текста на мелкие части, которые можно обработать машинными методами.
Можно использовать разные алгоритмы токенизации. В данном задании мы будем использовать `WordPunctTokenizer`.

In [9]:
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()

text = 'Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...'

print("before:", text,)
print("after:", tokenizer.tokenize(text.lower()))

before: Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...
after: ['здраствуйте', '.', 'я', ',', 'кирилл', '.', 'хотел', 'бы', 'чтобы', 'вы', 'сделали', 'игру', ',', '3д', '-', 'экшон', 'суть', 'такова', '...']


__Задание:__ реализуйте функцию ниже.

In [10]:
def preprocess(text: str, tokenizer) -> str:
    """
    Данная функция принимает на вход текст, 
    а возвращает тот же текст, но с пробелами между каждым токеном
    """
    
    return ' '.join(tokenizer.tokenize(text.lower()))

In [11]:
assert preprocess(text, tokenizer) == 'здраствуйте . я , кирилл . хотел бы чтобы вы сделали игру , 3д - экшон суть такова ...'

__Задание:__ токенизируйте `'title'` и `'description'` в `train` и `test`.

In [12]:
X_train = np.array([[preprocess(title, tokenizer), preprocess(description, tokenizer)] for title, description in tqdm(X_train)])
X_test = np.array([[preprocess(title, tokenizer), preprocess(description, tokenizer)] for title, description in tqdm(X_test)])

100%|██████████| 9000/9000 [00:00<00:00, 26326.43it/s]


In [13]:
assert X_train[5][0] == '1 - к квартира , 33 м² , 4 / 5 эт .'
assert X_train[10][1] == 'продам иж планета 3 , 76 год , ( стоит на старом учёте , документы утеряны ) на ходу , хорошее состояние , все интересующие вопросы по телефону ( с родной коляской на 3 тысячи дороже ) . торга не будет .'
assert X_test[2][0] == 'фара правая toyota rav 4 галоген 2015 - 19'
assert X_test[2][1] == 'фара правая для toyota rav4 2015 / оригинальный номер : 8113042650 / тойота рав4 тоета рав 4 / производитель : toyota / состояние : отличное без дефектов ! / комментарий : после 2015 не ксенон галоген + диод / пожалуйста , уточняйте соответствие вашего заказа изображенному на фото . / звоните уточняйте по наличию предоставляется время на проверку детали / отправляем в регионы рф транспортными компаниями / . / всегда включен вайбер вацап по вопросам !/ дополнительное фото по запросу'

### BOW (3 балла)

Один из традиционных подходов -- построение bag of words.

Метод состоит в следующем:

 - Составить словарь самых часто встречающихся слов в `train data`
 - Для каждого примера из `train` посчитать, сколько раз каждое слово из словаря в нём встречается


 В `sklearn` есть `CountVectorizer`, но в этом задании его использовать нельзя.

__Задание:__ создайте словарь, где каждому токену соответствует количество раз, которое оно встретилось в `X_train`.

In [14]:
tokens = map(str, 
    np.concatenate(
    [string.split() for string in np.concatenate(X_train, axis = None)], # куча токенов из выборки
               axis = None)
   )

In [15]:
from collections import Counter

tokens_cnt = Counter(tokens)

In [16]:
assert tokens_cnt['сапоги'] == 454

__Задание:__ выведите 10 самых частотных и 10 самых редких токенов.

In [17]:
# Самые частотные:
[k for k, v in sorted(tokens_cnt.items(), key=lambda item: -item[1])[:10]]

['/', ',', '.', '-', 'в', 'и', 'на', './', ':', 'с']

In [18]:
# Самые редкие:
[k for k, v in sorted(tokens_cnt.items(), key=lambda item: item[1])[:10]]

['iqmac',
 'ядерным',
 'ресурсоемкие',
 'быстродействием',
 'кинематографическому',
 'ирис',
 'саженец',
 'корень',
 'зубчаниновка',
 'боярышник']

__Задание:__ оставьте в словаре только топ-10000 самых частотных токенов, также создайте отдельный список из этих слов.

In [19]:
tokens_cnt = {k : v for k, v in sorted(tokens_cnt.items(), key=lambda item: -item[1])[:10000]}
tokens_list = list(tokens_cnt.keys())

__Задание:__ реализуйте функцию, которая переводит текст в вектор из чисел. То есть каждому токену из списка токенов сопоставляется количество раз, которое он встретился в тексте.

In [20]:
def text_to_bow(text: str, tokens_list: list) -> np.array:
    """
    Возвращает вектор, где для каждого слова из словаря
    указано количество его употреблений в предложении
    input: строка, список токенов
    output: вектор той же размерности, что и список токенов
    """
    
    text = text.split()
    counter = dict(Counter(text))
    
    return np.array([counter.get(word, 0) for word in tokens_list]) # возвращаем value из словаря (если есть 
                                                                    # соответствующий key, иначе - 0)

In [21]:
example_text = text_to_bow("сдаётся уютный , тёплый гараж для стартапов в ml", tokens_list)

assert np.allclose(example_text.mean(), 0.0008)

__Задание:__ а теперь реализуйте функцию, которая преобразует наш датасет и каждому тексту из `'description'` сопоставляет вектор.

In [22]:
def descr_to_bow(items: np.array, tokens_list: list) -> np.array:
    """ Для каждого описания товара возвращает вектор его bow """
    
    descriptions = items[:,1]
    return np.array([text_to_bow(text, tokens_list) for text in tqdm(descriptions)])

In [23]:
X_train_bow = descr_to_bow(X_train, tokens_list)
X_test_bow = descr_to_bow(X_test, tokens_list)

100%|██████████| 9000/9000 [00:25<00:00, 357.69it/s]


In [24]:
assert X_train_bow.shape == (21000, 10000), X_test_bow.shape == (9000, 10000)
assert 0.005 < X_train_bow.mean() < 0.006
assert 0.005 < X_test_bow.mean() < 0.006

### Логистическая регрессия и SVC (0.5 балла)


Теперь описание каждого товара представлено, как точка в многомерном пространстве.
Очень важно запомнить эту идею: дальше мы будем рассматривать разные способы перехода от текста к точке в пространстве.

Для BOW каждое измерение в пространстве -- какое-то слово.
Мы предполагаем, что текст описывается набором каких-то популярных слов, которые в нём встречаются, а близкие по смыслу тексты будут использовать одинаковые слова.

Обучите логистическую регрессию и SVM с линейным ядром (`sklearn.svm.LinearSVC` или `sklearn.svm.SVC(kernel='linear')`) с базовыми параметрами. При необходимости можете увеличить максимальное число итераций. В качестве `random_state` возьмите 13.

_Подсказка: для того, чтобы было проще обучать, можно использовать [разреженные матрицы](https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D0%B7%D1%80%D0%B5%D0%B6%D0%B5%D0%BD%D0%BD%D0%B0%D1%8F_%D0%BC%D0%B0%D1%82%D1%80%D0%B8%D1%86%D0%B0) - многие модели из `sklearn` умеют с ними работать. Соответствующий модуль из `scipy`: [scipy.sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html). Нетрудно заметить, что в полученных BOW-матрицах очень много нулей. Если хранить в памяти только ненулевые элементы, можно сильно оптимизировать вычисления. Можете в этом убедиться:_

In [25]:
print('Train array in memory (raw): {:.3f} Mb'.format(X_train_bow.nbytes * 1e-6))

from scipy.sparse import csr_matrix
X_train_bow_csr = csr_matrix(X_train_bow)
print('Train array in memory (compressed): {:.3f} Mb'.format(
    (X_train_bow_csr.data.nbytes + X_train_bow_csr.indptr.nbytes + X_train_bow_csr.indices.nbytes) * 1e-6)
)

Train array in memory (raw): 1680.000 Mb
Train array in memory (compressed): 8.606 Mb


In [26]:
from sklearn.metrics import accuracy_score

In [27]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(random_state = 13, max_iter = 500)

logreg.fit(X_train_bow_csr, y_train)
y_pred = logreg.predict(X_test_bow)
print('Accuracy for LogReg: {}'.format(accuracy_score(y_test, y_pred)))

assert accuracy_score(y_test, y_pred) > 0.695

Accuracy for LogReg: 0.6966666666666667


In [28]:
from sklearn.svm import LinearSVC
svc = LinearSVC(random_state = 13, max_iter = 100000) # на 100000 итераций сошлось :)

svc.fit(X_train_bow_csr, y_train)
y_pred = svc.predict(X_test_bow)
print('Accuracy for SVC: {}'.format(accuracy_score(y_test, y_pred)))

assert accuracy_score(y_test, y_pred) > 0.68

Accuracy for SVC: 0.6842222222222222


### Модификация признаков (2 балла)

Прибавьте к соответствующим BOW-векторам BOW-вектора для `'title'` товара с некоторым весом. Изменится ли качество? Как вы можете это объяснить?

In [29]:
def title_to_bow(items: np.array, tokens_list: list) -> np.array:
    """ Для каждого заголовка товара возвращает вектор его bow """
    
    titles = items[:,0]
    return np.array([text_to_bow(text, tokens_list) for text in titles])

X_train_bow_title = X_train_bow + title_to_bow(X_train, tokens_list)
X_test_bow_title = X_test_bow + title_to_bow(X_test, tokens_list)

X_train_bow_title_csr = csr_matrix(X_train_bow_title)

In [30]:
logreg = LogisticRegression(random_state = 13, max_iter = 1000)
logreg.fit(X_train_bow_title_csr, y_train)
y_pred = logreg.predict(X_test_bow_title)
print('Accuracy for LogReg (titles included): {}'.format(accuracy_score(y_test, y_pred)))

svc = LinearSVC(random_state = 13, max_iter = 100000)
svc.fit(X_train_bow_title_csr, y_train)
y_pred = svc.predict(X_test_bow_title)
print('Accuracy for SVC (titles included): {}'.format(accuracy_score(y_test, y_pred)))

Accuracy for LogReg (titles included): 0.7787777777777778
Accuracy for SVC (titles included): 0.7535555555555555


**Качество повысилось.** Это можно объяснить, в частности, тем, что не все позиции содержат описание (или описание, точно передающее сущность лота), тогда как заголовок присутствует у всех позиций. Таким образом, после добавления заголовков в модель ее информированность повысилась, что привело к повышению качества.

Нормализуйте данные с помощью `MinMaxScaler` или `MinAbsScaler` перед обучением. Что станет с качеством и почему?

In [31]:
from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()

X_train_bow_csr_scaled = scaler.fit_transform(X_train_bow_csr)
X_test_bow_scaled = scaler.fit_transform(X_test_bow)

In [32]:
logreg = LogisticRegression(random_state = 13, max_iter = 1000)
logreg.fit(X_train_bow_csr_scaled, y_train)
y_pred = logreg.predict(X_test_bow_scaled)
print('Accuracy for LogReg (scaled): {}'.format(accuracy_score(y_test, y_pred)))

svc = LinearSVC(random_state = 13, max_iter = 100000)
svc.fit(X_train_bow_csr_scaled, y_train)
y_pred = svc.predict(X_test_bow_scaled)
print('Accuracy for SVC (scaled): {}'.format(accuracy_score(y_test, y_pred)))

Accuracy for LogReg (scaled): 0.6558888888888889
Accuracy for SVC (scaled): 0.689


Масштабирование данных не дало ощутимого результата (accuracy для логистической регрессии снизилось, для SVC &mdash; слегка повысилось). Это логично, поскольку масштабирование сохраняет относительное расстояние между текстами и не добавляет информации, например, о том, какие слова должны обладать большим весом (как это делает метод TF-IDF), из-за чего модель не претерпевает значительных изменений.

Почему в данном случае использовать `StandardScaler` &mdash; не очень хорошая идея?

Потому что признаки станут несравнимыми. Условно, до обработки столбцы числа вхождений слов "продам" и "гараж" содержат значения в одинаковых единицах измерения (штуках), тогда как после обработки с учетом различий в дисперсии и среднем они содержат абстрактные величины. Потенциально, это может привести к проблемам со сходимостью модели, поскольку искомые веса будут иметь разный порядок. К тому же, в некоторых случаях стандартное отклонение равняется нулю (см. код ниже), отчего обработка StandardScaler становится невозможной.

In [33]:
min(np.std(X_train_bow, axis = 0))

0.0

### Иная предобработка (1 балл)

**На выбор**:

- **либо** обучите модели, используя для предобработки токенизатор и лемматизатор `pymystem3.Mystem`.
- **либо** добавьте к предобработке стэмминг.

Сравните полученное сейчас качество с полученным ранее и сделайте вывод.

**План действий:**
- Избавимся от знаков препинания, которые в нашем случае не должны улучшать качество модели (вряд ли грустный смайлик увеличивает вероятность попадания товара в категорию "Биологически активные добавки").
- Лемматизируем выборку: мы же все-таки Авито изучаем!
- Будем работать со склеенным датасетом (title + description);
- Воспользуемся инструментом SciKit &mdash; CountVectorizer.

![Мем категории Б](https://i.ibb.co/fD3x3Ls/d9cc544fa4b7275a7d79474630b2f981.jpg)

In [34]:
from nltk.tokenize import word_tokenize
from pymystem3 import Mystem

mystem = Mystem(entire_input = False)

X_train_lemm = [' '.join(mystem.lemmatize(' '.join([title, description]))) for title, description in tqdm(X_train)]
X_test_lemm = [' '.join(mystem.lemmatize(' '.join([title, description]))) for title, description in tqdm(X_test)]

X_train_lemm[:5]

100%|██████████| 9000/9000 [00:20<00:00, 441.13it/s]


['сапог размер новый сапог размер новый',
 'светильник потолочный swarovski светильник потолочный swarovski штука цена за штука в эксплуатация год продаваться в связь со смена интерьер в квартира',
 'iphone plus red красный в наличие данный цена только для подписчик instagram iqmac новый красный айфон plus в наличие это элегантный и мощный смартфон который готовый в полный мера раскрывать новый возможность ios аппарат с ядерный процессор и гб озу с легкость решать самый ресурсоемкий задача позволять наслаждаться быстродействие тяжелый приложение и игра на дюймовый дисплей аппарат получать экран как у ipad pro так что картинка теперь соответствовать кинематографический стандарт',
 'пион ирис ромашка рассада пион куст р более шт саженец корень расти у мы более год розовый бордовый и белый на фото цветок п зубчаниновка либо пл революция быть ирис ромашка клубника боярышник и ирга',
 'кофта состояние отличный']

In [35]:
import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords
stop_words = stopwords.words('russian') # русские стоп-слова

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ivanskv/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:
from sklearn.feature_extraction.text import CountVectorizer
cnt_vec = CountVectorizer(stop_words = stop_words)

X_train_lemm_bow = cnt_vec.fit_transform(X_train_lemm)
X_test_lemm_bow = cnt_vec.transform(X_test_lemm)

In [37]:
logreg = LogisticRegression(random_state = 13, max_iter = 1000)
logreg.fit(X_train_lemm_bow, y_train)
y_pred = logreg.predict(X_test_lemm_bow)
print('Accuracy for LogReg (lemmatized): {}'.format(accuracy_score(y_test, y_pred)))

svc = LinearSVC(random_state = 13, max_iter = 100000)
svc.fit(X_train_lemm_bow, y_train)
y_pred = svc.predict(X_test_lemm_bow)
print('Accuracy for SVC (lemmatized): {}'.format(accuracy_score(y_test, y_pred)))

Accuracy for LogReg (lemmatized): 0.7957777777777778
Accuracy for SVC (lemmatized): 0.7833333333333333


Получили улучшение моделей по сравнению с полученными в предыдущем задании. На качество повлияло избавление от пунктуации, лемматизация, а также отсутствие ограничения на длину словаря.

### TF-IDF (5 баллов)

Не все слова полезны одинаково, давайте попробуем [взвесить](http://tfidf.com/) их, чтобы отобрать более полезные.


> TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).
> 
> IDF(t) = log_e(Total number of documents / Number of documents with term t in it).


В `sklearn` есть `TfidfVectorizer`, но в этом задании его использовать нельзя. Для простоты посчитайте общий tf-idf для `'title'` и `'description'` (то есть каждому объекту надо сопоставить вектор, где как документ будет рассматриваться конкатенация `'title'` и `'description'`).

__Задание:__ составьте словарь, где каждому слову из изначального списка будет соответствовать количество документов из `train`-части, где это слово встретилось.

In [38]:
# токены для каждого "документа" (title + description) из X_train:
doc_tok = [string.split() for string in map(' '.join, X_train)] 

# Вспомогательная функция. Для данного слова считает число текстов, содержащих его. 
def doc_count(word: str):
    return sum([word in document for document in doc_tok])


In [39]:
word_document_cnt = {word : doc_count(word) for word in tqdm(tokens_list)}

100%|██████████| 10000/10000 [03:55<00:00, 42.50it/s]


In [40]:
assert word_document_cnt['размер'] == 2839

__Задание:__ реализуйте функцию, где тексту в соответствие ставится tf-idf вектор. Для вычисления IDF также необходимо число документов в `train`-части (параметр `n_documents_total`).

In [42]:
def text_to_tfidf(text: str, word_document_cnt: dict, tokens_list: list, n_documents_total: int) -> np.array:
    """
    Возвращает вектор, где для каждого слова из словаря
    указан tf-idf
    """
    
    bow = text_to_bow(text, tokens_list)
    n_words = len(text.split())
    
    return np.array(
        [(bow[i] / n_words) * np.log(n_documents_total / word_document_cnt[word]) for i, word in enumerate(tokens_list)]
    )

In [43]:
example_text = text_to_tfidf(
    'сдаётся уютный , тёплый гараж для стартапов в ml',
    word_document_cnt,
    tokens_list,
    n_documents_total=len(X_train)
)

assert 0.0003 < example_text.mean() < 0.0004

__Задание:__ а теперь реализуйте функцию, которая преобразует наш датасет и для каждого объекта сопоставляет вектор tf-idf. В качестве текстов используйте конкатенацию `'title'` и `'description'`.

In [44]:
def items_to_tfidf(items: np.array, word_document_cnt: dict, tokens_list: list, n_documents_total: int) -> np.array:
    """
    Для каждого товара возвращает его tf-idf вектор
    """
    return np.array(
        [text_to_tfidf(title + ' ' + description, word_document_cnt, tokens_list, n_documents_total) for title, description in tqdm(items)]
    )

In [45]:
X_train_tfidf = items_to_tfidf(X_train, word_document_cnt, tokens_list, len(X_train))
X_test_tfidf = items_to_tfidf(X_test, word_document_cnt, tokens_list, len(X_train))

100%|██████████| 9000/9000 [03:10<00:00, 47.23it/s]


In [46]:
assert X_train_tfidf.shape == (21000, 10000), X_test_tfidf.shape == (9000, 10000)
assert 0.0002 < X_train_tfidf.mean() < 0.0004
assert 0.0002 < X_test_tfidf.mean() < 0.0004

__Задание:__ обучите логистическую регрессию и SVC, оцените качество (accuracy_score). Сделайте вывод.

In [47]:
X_train_tfidf_csr = csr_matrix(X_train_tfidf)

In [48]:
lr_model = LogisticRegression(random_state = 13, max_iter = 1000)
lr_model.fit(X_train_tfidf_csr, y_train)
print('Accuracy for LogReg (TF-IDF): {}'.format(accuracy_score(y_test, lr_model.predict(X_test_tfidf))))

assert accuracy_score(y_test, lr_model.predict(X_test_tfidf)) > 0.675

Accuracy for LogReg (TF-IDF): 0.6795555555555556


In [49]:
svc_model = LinearSVC(random_state = 13, max_iter = 100000)
svc_model.fit(X_train_tfidf_csr, y_train)
print('Accuracy for SVC (TF-IDF): {}'.format(accuracy_score(y_test, svc_model.predict(X_test_tfidf))))

assert accuracy_score(y_test, svc_model.predict(X_test_tfidf)) > 0.79

Accuracy for SVC (TF-IDF): 0.7942222222222223


### Word Vectors (4 балла)

Давайте попробуем другой подход -- каждому слову сопоставим какое-то векторное представление (эмбеддинг) - но достаточно маленькой размерности. Таким образом мы сильно уменьшим количество параметров в модели.

Почитать про это подробнее можно тут:

- https://habr.com/ru/company/ods/blog/329410/

Вектора мы возьмём уже готовые (обученные на текстах из интернета), так что наша модель будет знать некоторую дополнительную информацию о внешнем мире.

In [50]:
# !wget https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz

In [51]:
# !tar -xzf ru.tar.gz

In [52]:
!pip install gensim

In [53]:
import gensim
from gensim.models.wrappers import FastText

embedding_model = FastText.load_fasttext_format('ru.bin')

unable to import 'smart_open.gcs', disabling that module


In [54]:
# как мы видим, каждому слову данная модель сопоставляет вектор размерности 300

print(embedding_model['привет'].shape)
print(embedding_model['привет'])

(300,)
[ 0.02916384  0.02167605  0.05127367 -0.00971958  0.0465235  -0.03945766
  0.02737866  0.00638128 -0.03774629 -0.04257201 -0.00995653  0.02291315
 -0.02301722  0.06697998 -0.03674482 -0.02403202 -0.05404469  0.01372932
  0.00926399 -0.0013149   0.11941359 -0.022448    0.04011497  0.06980549
  0.00407011 -0.09384539  0.03050164 -0.02578281 -0.03525181 -0.06603175
  0.04752798  0.05874675  0.01983666  0.06092105 -0.00957561  0.08307806
 -0.01288903  0.04705157  0.02198839 -0.00649013 -0.0171444   0.03302203
  0.02124882 -0.01902875 -0.05235172  0.03458685 -0.01409259 -0.07477519
  0.01916078  0.02985001  0.0086322   0.03051201  0.02831862  0.04549561
  0.00761138 -0.05459622  0.09056009 -0.08807947 -0.05420396 -0.04793203
 -0.05672329 -0.03025264 -0.03024072 -0.05890108 -0.03137474  0.03292617
  0.05440779 -0.04548327 -0.07266086 -0.09327219  0.07247883  0.0111061
  0.01824225 -0.10570452  0.05110046 -0.04659343 -0.03277056 -0.00803401
 -0.03978698  0.00826598 -0.01074128  0.01843

__Задание:__ реализуйте функцию, выдающую эмбеддинг для предложения - как сумму эмбеддингов токенов.

In [55]:
def sentence_embedding(sentence: str, embedding_model) -> np.array:
    """
    Складывает вектора токенов строки sentence
    """
    
    embedding = np.sum(
        np.array([embedding_model[word] for word in sentence.split() if word in embedding_model]), 
        axis = 0
    )
    
    if embedding.shape == (300,):
        return embedding
    else:
        return np.zeros((300,)) # нужно, чтобы позиции без слов на русском возвращались как вектор, а не как float 0.0

Я представляю "проблемные" позиции (т. е. позиции, в описании которых нет ни одного слова, содержащегося в обученной модели) как нулевые векторы  из **прагматических соображений.** Если так не делать, такие строки будут закодированы как float 0.0, из-за чего нарушится размерность в np.array, и конечная embedded выборка будет не матрицей, а вектором векторов и чисел. Такой подход не идеален, поскольку слово &mdash; нулевой вектор близко к огромному количеству других слов, однако в рамках нашей задачи это допустимо: в выборке X_train + X_test мало "проблемных" позиций (в этом можно убедиться ниже), так что на качество конечной модели это не должно особо влиять. К тому же, не я один [использую](https://datascience.stackexchange.com/questions/32345/initial-embeddings-for-unknown-padding) этот метод, так что это немного снимает ответственность :)

In [56]:
assert sentence_embedding('сдаётся уютный , тёплый гараж для стартапов в ml', embedding_model).shape == (300,)
assert np.allclose(np.linalg.norm(sentence_embedding('сдаётся уютный , тёплый гараж для стартапов в ml', embedding_model)),2.6764746)

__Задание:__ сделайте все то же, что в предыдущих пунктах -- реализуйте функцию, которая преобразует данные, а затем обучите логистическую регрессию и SVM, оцените качество. Сделайте вывод, что работает лучше - модель, основанная на TF-IDF, или модель, обученная на предобученных эмбеддингах?

In [57]:
def items_to_embeddings(items: np.array, embedding_model) -> np.array:
    """
    Для каждого товара (title + description) возвращает его эмбеддинг
    """
    
    return np.array(
        [sentence_embedding(title + ' ' + description, embedding_model) for title, description in tqdm(items)]
    )

In [58]:
X_train_embed = items_to_embeddings(X_train, embedding_model)
X_test_embed = items_to_embeddings(X_test, embedding_model)

X_train_embed_csr = csr_matrix(X_train_embed)

100%|██████████| 9000/9000 [00:10<00:00, 886.45it/s]


Как и обещал, считаем количество "проблемных" позиций в полной выборке:

In [59]:
a = sum(np.invert([row.all() for row in X_train_embed]))
b = sum(np.invert([row.all() for row in X_test_embed]))

print('Проблемных позиций в тренировочной выборке: {}'.format(a))
print('Проблемных позиций в тестовой выборке: {}'.format(b))

Проблемных позиций в тренировочной выборке: 8
Проблемных позиций в тестовой выборке: 3


Наконец, прогоняем регрессии:

In [60]:
X_train_embed

array([[-0.03451314,  0.01240876, -0.01735052, ..., -0.2620967 ,
         0.23882353, -0.33055413],
       [ 0.12331572, -0.32799682,  0.37786162, ...,  0.39820388,
        -0.15178642,  0.08611983],
       [-0.57603192,  0.53449881,  0.86583161, ...,  1.0049262 ,
         0.94092357, -1.32305908],
       ...,
       [ 2.52035904,  2.019104  ,  3.0233376 , ...,  2.80345416,
        -1.74237657,  0.5877378 ],
       [ 0.02273147, -0.18537723, -0.03700086, ...,  0.08827318,
        -0.13819212, -0.1487271 ],
       [-0.54405159,  0.71978855,  1.08315849, ...,  0.60593623,
         1.19675696,  0.71603405]])

In [61]:
from sklearn.svm import LinearSVC

In [62]:
# Прогоняется очень долго, так что жертвуем сходимостью ради экономии времени.

logreg = LogisticRegression(random_state = 13, max_iter = 1000)
logreg.fit(X_train_embed_csr, y_train)
y_pred = logreg.predict(X_test_embed)
print('Accuracy for LogReg (embedded): {}'.format(accuracy_score(y_test, y_pred)))

svc = LinearSVC(random_state = 13, max_iter = 1000)
svc.fit(X_train_embed_csr, y_train)
y_pred = svc.predict(X_test_embed)
print('Accuracy for SVC (embedded): {}'.format(accuracy_score(y_test, y_pred)))

Accuracy for LogReg (embedded): 0.5828888888888889
Accuracy for SVC (embedded): 0.577


Как видно, качество моделей значительно ухудшилось (по сравнению с TF-IDF случаем: 0.68 для LogReg и 0.794 для SVC).

### Что дальше? (8 баллов)

Для получения максимальной оценки вам нужно решить любые 2 пункта. Решение каждого пункта даст вам полтора балла:

1. Реализовать n-gram модели текстовой классификации (__2 балла__) <font color='green'>&#10003;</font> 

2. Поработать с другими эмбеддингами для слов (например `word2vec` или `GloVe`) (__2 балла__) <font color='red'>&#10003;</font> 

3. Применить другие способы токенизации (например, `pymorphy2`, `spaCy`) и в целом предобработки данных (стоп-слова, стэмминг, лемматизация) (__2 балла__) <font color='green'>&#10003;</font> 

4. Добиться качества > 0.82 на тестовых данных (попробуйте другие токенизаторы, предобработку текста, и любые другие идеи, которые вам придут в голову) (__2 балла__) <font color='green'>&#10003;</font> 

Снабжайте код пояснениями и графиками.
Обязательно необходимо написать вывод по каждому пункту, который вы реализуете.

#### Пункт №1

Реализуем n-gram модели средствами SciKitLearn. Будем использовать униграммы, биграммы и триграммы на лемматизированной выборке.

In [63]:
cnt_vec_ngram = CountVectorizer(stop_words = stop_words, ngram_range=(1, 3))

X_train_ngram_bow = cnt_vec_ngram.fit_transform(X_train_lemm)
X_test_ngram_bow = cnt_vec_ngram.transform(X_test_lemm)

In [64]:
# Опять жертвуем сходимостью ради экономии времени.

logreg = LogisticRegression(random_state = 13, max_iter = 40)
logreg.fit(X_train_ngram_bow, y_train) 
y_pred = logreg.predict(X_test_ngram_bow)
print('Accuracy for LogReg (n_grams 1 to 3): {}'.format(accuracy_score(y_test, y_pred)))

svc = LinearSVC(random_state = 13, max_iter = 40)
svc.fit(X_train_ngram_bow, y_train)
y_pred = svc.predict(X_test_ngram_bow)
print('Accuracy for SVC (n_grams 1 to 3): {}'.format(accuracy_score(y_test, y_pred)))

Accuracy for LogReg (n_grams 1 to 3): 0.7833333333333333
Accuracy for SVC (n_grams 1 to 3): 0.7874444444444444


Добавление n-gram в выборку практически не изменило качество модели по сравнению со случаем использования униграмм (с качеством 0.7951 для LogReg и 0.784 для SVC). При этом легко убедиться, что униграммы все еще составляют основу качества модели: если исключить их из данных, качество значительно уменьшится:

In [65]:
cnt_vec_ngram = CountVectorizer(stop_words = stop_words, ngram_range=(3, 3))

X_train_ngram_bow = cnt_vec_ngram.fit_transform(X_train_lemm)
X_test_ngram_bow = cnt_vec_ngram.transform(X_test_lemm)

logreg = LogisticRegression(random_state = 13, max_iter = 20)
logreg.fit(X_train_ngram_bow, y_train) 
y_pred = logreg.predict(X_test_ngram_bow)
print('Accuracy for LogReg (n_grams 1 to 3): {}'.format(accuracy_score(y_test, y_pred)))

svc = LinearSVC(random_state = 13, max_iter = 20) 
svc.fit(X_train_ngram_bow, y_train)
y_pred = svc.predict(X_test_ngram_bow)
print('Accuracy for SVC (n_grams 1 to 3): {}'.format(accuracy_score(y_test, y_pred)))

Accuracy for LogReg (n_grams 1 to 3): 0.4151111111111111
Accuracy for SVC (n_grams 1 to 3): 0.44766666666666666


Видимо, это связано с тем, что в рамках нашей классификационной задачи б__о__льшую роль играют отдельные слова, нежели словосочетания: в описании товара всегда присутсвует его наименование, которое, по сути, и составляет значительную часть ценной информации, получаемой моделью.

#### Пункт №2

Загрузим предобученную на НКРЯ и Википедии Word2Vec модель с сайта [RusVectōrēs](https://rusvectores.org/ru/models/) (архив ruwikiruscorpora_upos_skipgram_300_2_2019).

In [66]:
# !wget http://vectors.nlpl.eu/repository/20/182.zip

In [67]:
import zipfile

with zipfile.ZipFile('182.zip', 'r') as archive:
    stream = archive.open('model.bin')
    embedding_model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True) # Подгружаем модель

Заметим, что в данной модели к каждому слову из словаря "приклеено" наименование его части речи:

In [68]:
'вектор' in embedding_model

False

In [69]:
'вектор_NOUN' in embedding_model

True

Поэтому перед использованием эмбеддинга необходимо привести нашу выборку к такому же виду. RusVectōrēs предоставляет [скрипт](https://github.com/akutuzov/webvectors/blob/master/preprocessing/rus_preprocessing_mystem.py) для трансформации предложений в набор POS-тэгнутых токенов (от POS &mdash; Part of Speech). Скрипт ниже немного преобразован и позволяет игнорировать нерусскоязычные слова, которые могут встречаться в выборке.

In [70]:
m = Mystem()

def tag_mystem(text='Текст нужно передать функции в виде строки!', mapping=None, postags=True):
    # если частеречные тэги не нужны (например, их нет в модели), выставьте postags=False
    # в этом случае на выход будут поданы только леммы

    processed = m.analyze(text)
    tagged = []
    for w in processed:
        try:
            lemma = w["analysis"][0]["lex"].lower().strip()
            pos = w["analysis"][0]["gr"].split(',')[0]
            pos = pos.split('=')[0].strip()
            if mapping:
                if pos in mapping:
                    pos = mapping[pos]  # здесь мы конвертируем тэги
                else:
                    pos = 'X'  # на случай, если попадется тэг, которого нет в маппинге
            tagged.append(lemma.lower() + '_' + pos)
        except (KeyError, IndexError):
            continue  # я здесь пропускаю знаки препинания, но вы можете поступить по-другому
    if not postags:
        tagged = [t.split('_')[0] for t in tagged]
    return ' '.join(tagged)

In [71]:
import requests

mapping_url = 'https://raw.githubusercontent.com/akutuzov/universal-pos-tags/4653e8a9154e93fe2f417c7fdb7a357b7d6ce333/ru-rnc.map'

mystem2upos = {} # маппинг для конвертации Mystem-тегов в Universal POS Tags
                 # https://universaldependencies.org/u/pos/all.html

r = requests.get(mapping_url, stream=True)
for pair in r.text.split('\n'):
    pair = pair.split()
    if len(pair) > 1:
        mystem2upos[pair[0]] = pair[1]

Посмотрим, как работает эта функция:

In [72]:
tag_mystem('Мастер жрет сам.', mapping = mystem2upos)

'мастер_NOUN жрать_VERB сам_DET'

Преобразуем нашу выборку:

In [73]:
X_train_tagged = [tag_mystem(title + ' ' + description, mapping = mystem2upos) for title, description in tqdm(X_train)]
X_test_tagged = [tag_mystem(title + ' ' + description, mapping = mystem2upos) for title, description in tqdm(X_test)]

X_train_tagged[:2]

100%|██████████| 9000/9000 [00:21<00:00, 416.14it/s]


['сапог_NOUN размер_NOUN новый_ADJ сапог_NOUN размер_NOUN новый_ADJ',
 'светильник_NOUN потолочный_ADJ светильник_NOUN потолочный_ADJ штука_NOUN цена_NOUN за_ADP штука_NOUN в_ADP эксплуатация_NOUN год_NOUN продаваться_VERB в_ADP связь_NOUN со_ADP смена_NOUN интерьер_NOUN в_ADP квартира_NOUN']

Теперь можно воспользоваться уже заданной функцией `sentence_embedding`, чтобы прогнать эмбеддинг.

In [74]:
X_train_tagged_embed = [sentence_embedding(document, embedding_model) for document in tqdm(X_train_tagged)]
X_test_tagged_embed = [sentence_embedding(document, embedding_model) for document in tqdm(X_test_tagged)]

X_train_tagged_embed_csr = csr_matrix(X_train_tagged_embed)

100%|██████████| 9000/9000 [00:00<00:00, 9614.45it/s]


Наконец, гоняем регрессии!

In [75]:
logreg = LogisticRegression(random_state = 13, max_iter = 1000)
logreg.fit(X_train_tagged_embed_csr, y_train) 
y_pred = logreg.predict(X_test_tagged_embed)
print('Accuracy for LogReg (RusVectōrēs embedded): {}'.format(accuracy_score(y_test, y_pred)))

svc = LinearSVC(random_state = 13, max_iter = 1000) 
svc.fit(X_train_tagged_embed_csr, y_train)
y_pred = svc.predict(X_test_tagged_embed)
print('Accuracy for SVC (RusVectōrēs embedded): {}'.format(accuracy_score(y_test, y_pred)))

Accuracy for LogReg (RusVectōrēs embedded): 0.7296666666666667
Accuracy for SVC (RusVectōrēs embedded): 0.7063333333333334


Результат неплохой: сравнится с TF-IDF моделью и сильно превосходит другой эмбеддинг, проведенный выше.

#### Пункт №3

В этот раз воспользуемся стеммингом и проверим, сравнится ли он с православной лемматизацимей.

![Лемматизация](https://upload.wikimedia.org/wikipedia/commons/0/00/Храм_Покрова_на_Нерли_в_Боголюбово_4.jpg)

In [76]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('russian')


In [77]:
def text_to_stem(text : str, stemmer):
    return [stemmer.stem(word) for word in text.split()]


def items_to_stem(items : np.array, stemmer):
    
    items_join = [title + ' ' + description for title, description in items]
    
    return [' '.join(text_to_stem(document, stemmer)) for document in tqdm(items_join)]


X_train_stem = items_to_stem(X_train, stemmer)
X_test_stem = items_to_stem(X_test, stemmer)

100%|██████████| 9000/9000 [00:25<00:00, 355.41it/s]


In [78]:
cnt_vec = CountVectorizer(stop_words = stop_words)

X_train_stem_bow = cnt_vec.fit_transform(X_train_stem)
X_test_stem_bow = cnt_vec.transform(X_test_stem)

In [79]:
logreg = LogisticRegression(random_state = 13, max_iter = 1000)
logreg.fit(X_train_stem_bow, y_train)
y_pred = logreg.predict(X_test_stem_bow)
print('Accuracy for LogReg (stemmatized): {}'.format(accuracy_score(y_test, y_pred)))

svc = LinearSVC(random_state = 13, max_iter = 100000)
svc.fit(X_train_stem_bow, y_train)
y_pred = svc.predict(X_test_stem_bow)
print('Accuracy for SVC (stemmatized): {}'.format(accuracy_score(y_test, y_pred)))

Accuracy for LogReg (stemmatized): 0.806
Accuracy for SVC (stemmatized): 0.7901111111111111


Стемминг показал себя лучше лемматизации (по сравнению с качеством 0.795 для LogReg и 0.784 для SVC). К тому же, стемминг работает быстрее лемматизации ("по определению"), так что win-win! Попробуем также объединить стемминг с TF-IDF:

#### Пункт №4

Добъемся качества $> 0.82$! В качестве основы используем `X_train_lemm` и `X_test_lemm`, полученные выше (напомню: там объединены колонки title и description и проведена лемматизация средствами Mystem с исключением знаков препинания). Теперь применим TF-IDF к этим предобработанным данным и прогоним логистическую регрессию и SVC:

In [84]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words = stop_words)

X_train_lemm_tfidf = tfidf.fit_transform(X_train_lemm)
X_test_lemm_tfidf = tfidf.transform(X_test_lemm)

In [85]:
logreg = LogisticRegression(random_state = 13, max_iter = 1000)
logreg.fit(X_train_lemm_tfidf, y_train)
y_pred = logreg.predict(X_test_lemm_tfidf)
print('Accuracy for LogReg (titles included, lemmatized, TF-IDF): {}'.format(accuracy_score(y_test, y_pred)))

svc = LinearSVC(random_state = 13, max_iter = 100000)
svc.fit(X_train_lemm_tfidf, y_train)
y_pred = svc.predict(X_test_lemm_tfidf)
print('Accuracy for SVC (titles included, lemmatized, TF-IDF): {}'.format(accuracy_score(y_test, y_pred)))

Accuracy for LogReg (titles included, lemmatized, TF-IDF): 0.7795555555555556
Accuracy for SVC (titles included, lemmatized, TF-IDF): 0.8302222222222222


Модель SVC на основе TF-IDF данных с предварительной лемматизацией дала качество 0.83. Заметим, что комбинация стемминга и TF-IDF дает аналогичные результаты:

In [86]:
X_train_stem_tfidf = tfidf.fit_transform(X_train_stem)
X_test_stem_tfidf = tfidf.transform(X_test_stem)

In [87]:
logreg = LogisticRegression(random_state = 13, max_iter = 1000)
logreg.fit(X_train_stem_tfidf, y_train)
y_pred = logreg.predict(X_test_stem_tfidf)
print('Accuracy for LogReg (titles included, stemmatized, TF-IDF): {}'.format(accuracy_score(y_test, y_pred)))

svc = LinearSVC(random_state = 13, max_iter = 100000)
svc.fit(X_train_stem_tfidf, y_train)
y_pred = svc.predict(X_test_stem_tfidf)
print('Accuracy for SVC (titles included, stemmatized, TF-IDF): {}'.format(accuracy_score(y_test, y_pred)))

Accuracy for LogReg (titles included, stemmatized, TF-IDF): 0.7794444444444445
Accuracy for SVC (titles included, stemmatized, TF-IDF): 0.8348888888888889


____
![Все!](https://batenka.ru/media/original_images/er11.jpg)